In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext import data
from torchtext.data import Field
from torchtext.data import BucketIterator

# from torch.utils.data import Dataset,DataLoader

import spacy
import numpy as np
import os
import random
import pandas as pd
import time
import math

from sklearn.model_selection import train_test_split
import json
import pathlib
import numpy as np
import gensim

import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from gensim import models
model = models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/IRE_PROJECT/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
train_dataset={'story':[],'labels':[]}
for line in open('/content/gdrive/My Drive/IRE_PROJECT/train_data.json'):
  data=json.loads(line)
  # print(data['story'])
  # print(data['labels'])
  train_dataset['story'].append(data['story'])
  train_dataset['labels'].append(data['labels'])
# train=json.loads(pathlib.Path('/content/train_data.json').read_text())

In [ ]:
validation_dataset={'story':[],'labels':[]}
for line in open('/content/gdrive/My Drive/IRE_PROJECT/validation_data.json'):
  data=json.loads(line)
  # print(data['story'])
  # print(data['labels'])
  validation_dataset['story'].append(data['story'])
  validation_dataset['labels'].append(data['labels'])
# train=json.loads(pathlib.Path('/content/train_data.json').read_text())

In [ ]:
train_df = pd.DataFrame(train_dataset, columns=["story", "labels"])
validation_df=pd.DataFrame(validation_dataset, columns=["story", "labels"])

In [ ]:
train_df

,story,labels
0,[Paris ( CNN ) -- Flamboyant fashion designer ...,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,"[( CNN ), -- I spent much of Sunday in touch w...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[New York ( CNN ) --, The head of the Internat...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,"[( CNN ) --, All the self-righteous huffing an...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
4,"[Los Angeles ( CNN ) --, Tour manager Paul Gon...","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
3235,"[( CNN ), -- South Korea will attempt a second...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
3236,"[( Mental Floss ) --, It 's hard to walk down ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3237,"[Yangon , Myanmar ( CNN ) -- Myanmar oppositio...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3238,[( CNN ) -- Not only is Sheikha Lubna Al Qasim...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [ ]:
validation_df

,story,labels
0,"[( CNN ) --, Sporting a black t-shirt proudly ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,[WASHINGTON ( CNN ) -- How does the American p...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, ..."
2,"[( CNN ) --, Basketball star Dennis Rodman see...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,"[( CNN ), -- Check with your airline before yo...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[RIYADH , Saudi Arabia --, Ahmad al Shayea is ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
...,...,...
355,"[( CNN ) --, The World Health Organization rai...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
356,"[Dali , Guizhou , China ( CNN ) -- Shi Wenchan...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
357,"[Cairo , Egypt ( CNN ) -- One man died and ano...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
358,[London ( CNN ) -- The cleanup operation conti...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [ ]:
weights = model.vectors
weights = np.insert(weights, 0,np.zeros(300),axis = 0)
weights = np.insert(weights, 0,np.zeros(300),axis = 0)
weights = torch.FloatTensor(weights)

In [ ]:
word2id = {model.index2word[i]:i+2 for i in range(len(model.index2word))}
word2id['<pad>'] = 0
word2id['<unk>'] = 1

In [ ]:
class Dataset():
    def __init__(self, data_list):
        self._data = data_list
    def __len__(self):
        return len(self._data)
        
    def __call__(self, batch_size, shuffle = True):
        max_len = len(self)
        
        indices = [i for i in range(0,max_len,batch_size)]
        
        if shuffle:
            indices = np.random.randint(0,max_len-1,math.ceil(max_len/batch_size))
            
        batchs = [self._data[index:index + batch_size] for index in indices]
        return batchs
    
    def __getitem__(self, index):
        return self._data.iloc[index,:].to_numpy()
    
class DataLoader():
    def __init__(self, dataset, batch_size = 1, shuffle = True):
        assert isinstance(dataset, Dataset)
        assert len(dataset) >= batch_size
        self.shuffle = shuffle
        self.dataset = dataset
        self.batch_size = batch_size
    def __iter__(self):
        return iter(self.dataset(self.batch_size, self.shuffle))
    
    

In [ ]:
class RNN_RNN(nn.Module):
    def __init__(self, embed=None):
        super(RNN_RNN, self).__init__()
        self.model_name = 'RNN_RNN'
        # self.args = args
        
        V = 3000002
        D = 300
        H = 200
        S = 10
        P_V = 100
        P_D = 50
        self.abs_pos_embed = nn.Embedding(P_V,P_D)
        self.rel_pos_embed = nn.Embedding(S,P_D)
        self.embed = nn.Embedding(V,D,padding_idx=0)
        if embed is not None:
            self.embed.weight.data.copy_(embed)

        self.word_RNN = nn.GRU(
                        input_size = D,
                        hidden_size = H,
                        batch_first = True,
                        bidirectional = True
                        )
        self.sent_RNN = nn.GRU(
                        input_size = 2*H,
                        hidden_size = H,
                        batch_first = True,
                        bidirectional = True
                        )
        self.fc = nn.Linear(2*H,2*H)

        # Parameters of Classification Layer
        self.content = nn.Linear(2*H,1,bias=False)
        self.salience = nn.Bilinear(2*H,2*H,1,bias=False)
        self.novelty = nn.Bilinear(2*H,2*H,1,bias=False)
        self.abs_pos = nn.Linear(P_D,1,bias=False)
        self.rel_pos = nn.Linear(P_D,1,bias=False)
        self.bias = nn.Parameter(torch.FloatTensor(1).uniform_(-0.1,0.1))

    def max_pool1d(self,x,seq_lens):
        # x:[N,L,O_in]
        out = []
        for index,t in enumerate(x):
            #print(index)
            t = t[:seq_lens[index],:]
            t = torch.t(t).unsqueeze(0)
            out.append(F.max_pool1d(t,t.size(2)))
            # print(out[-1])
        out = torch.cat(out).squeeze(2)
        return out
        
    def avg_pool1d(self,x,seq_lens):
        # x:[N,L,O_in]
        out = []
        for index,t in enumerate(x):
            t = t[:seq_lens[index],:]
            t = torch.t(t).unsqueeze(0)
            out.append(F.avg_pool1d(t,t.size(2)))
        
        out = torch.cat(out).squeeze(2)
        return out


    def pad_doc(self,words_out,doc_lens):
        pad_dim = words_out.size(1)
        max_doc_len = max(doc_lens)
        sent_input = []
        start = 0
        for doc_len in doc_lens:
            stop = start + doc_len
            valid = words_out[start:stop]                                       # (doc_len,2*H)
            start = stop
            if doc_len == max_doc_len:
                sent_input.append(valid.unsqueeze(0))
            else:
                pad = Variable(torch.zeros(max_doc_len-doc_len,pad_dim))
                #if self.args.device is not None:
                pad = pad.cuda()
                sent_input.append(torch.cat([valid,pad]).unsqueeze(0))          # (1,max_len,2*H)
        sent_input = torch.cat(sent_input,dim=0)                                # (B,max_len,2*H)
        return sent_input

    def forward(self,x,doc_lens):
        sent_lens = torch.sum(torch.sign(x),dim=1).data 
        x = self.embed(x)                                                      # (N,L,D)
        # word level GRU
        H = 200

        x = self.word_RNN(x)[0]                                                 # (N,2*H,L)
        # print(x.size() , sent_lens)
        
        #word_out = self.avg_pool1d(x,sent_lens)
        word_out = self.max_pool1d(x,sent_lens)
        # make sent features(pad with zeros)
        x = self.pad_doc(word_out,doc_lens)
        # print(x.size())
        # sent level GRU
        sent_out = self.sent_RNN(x)[0]                                           # (B,max_doc_len,2*H)
        #docs = self.avg_pool1d(sent_out,doc_lens)                               # (B,2*H)
        # print(sent_out.size(),doc_lens)

        docs = self.max_pool1d(sent_out,doc_lens)                                # (B,2*H)
        probs = []
        for index,doc_len in enumerate(doc_lens):
            valid_hidden = sent_out[index,:doc_len,:]                            # (doc_len,2*H)
            doc = F.tanh(self.fc(docs[index])).unsqueeze(0)
            s = Variable(torch.zeros(1,2*H))
            # if self.args.device is not None:
            s = s.cuda()
            for position, h in enumerate(valid_hidden):
                h = h.view(1, -1)                                                # (1,2*H)
                # get position embeddings
                abs_index = Variable(torch.LongTensor([[position]]))
                #if self.args.device is not None:
                
                abs_index = abs_index.cuda()
                abs_features = self.abs_pos_embed(abs_index).squeeze(0)
                
                rel_index = int(round((position + 1) * 9.0 / doc_len))
                rel_index = Variable(torch.LongTensor([[rel_index]]))
                # if self.args.device is not None:
                rel_index = rel_index.cuda()
                rel_features = self.rel_pos_embed(rel_index).squeeze(0)
                
                # classification layer
                content = self.content(h) 
                salience = self.salience(h,doc)
                novelty = -1 * self.novelty(h,F.tanh(s))
                abs_p = self.abs_pos(abs_features)
                rel_p = self.rel_pos(rel_features)
                prob = F.sigmoid(content + salience + novelty + abs_p + rel_p + self.bias)
                s = s + torch.mm(prob,h)
                probs.append(prob)
        return torch.cat(probs).squeeze()


In [ ]:
net = RNN_RNN(weights)
net.cuda()

RNN_RNN(
  (abs_pos_embed): Embedding(100, 50)
  (rel_pos_embed): Embedding(10, 50)
  (embed): Embedding(3000002, 300, padding_idx=0)
  (word_RNN): GRU(300, 200, batch_first=True, bidirectional=True)
  (sent_RNN): GRU(400, 200, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=400, out_features=400, bias=True)
  (content): Linear(in_features=400, out_features=1, bias=False)
  (salience): Bilinear(in1_features=400, in2_features=400, out_features=1, bias=False)
  (novelty): Bilinear(in1_features=400, in2_features=400, out_features=1, bias=False)
  (abs_pos): Linear(in_features=50, out_features=1, bias=False)
  (rel_pos): Linear(in_features=50, out_features=1, bias=False)
)

In [ ]:
import copy
class Batch:
  def __init__(self,batch,word2id):
    self.batch=copy.deepcopy(batch.to_numpy())
    self.word2id=word2id

  def preprocess(self):
    idx=0
    doc_lens = []
    features = []
    targets = []

    sent_list = []
    for doc in self.batch[:,0]:
        sent_list += doc
        doc_lens.append(len(doc))
        targets += list(self.batch[idx,1])
        idx+=1

    maxlen=0
    for sentence in sent_list:
      s=sentence.split()
      maxlen=max(maxlen,len(s))
  
    print(maxlen)

    for sentence in sent_list:
      words = sentence.strip().split()
      sentence = [self.word2id[word] if word in self.word2id else 1 for word in words]
      sentence += [0 for _ in range(maxlen - len(sentence))]
      features.append(sentence)

      

    # print(doc_lens)
    features = torch.LongTensor(features)    
    targets = torch.LongTensor(targets)
    
    return features,targets,doc_lens





In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
loss_sum = 0
min_eval_loss = float('Inf')
train_loader = DataLoader(Dataset(train_df),batch_size=16)
validation_loader=DataLoader(Dataset(validation_df),shuffle=False)
# for step, docs in enumerate(train_loader):
#         for doc in Batch(docs,word2id).preprocess():
 
counter=0
for epoch in range(5):
    for step, docs in enumerate(train_loader):
        features,targets,doc_lens = Batch(docs,word2id).preprocess()

        features,targets = Variable(features), Variable(targets.float())
        features = features.cuda()
        targets = targets.cuda()

        probs = net(features,doc_lens)
        loss = criterion(probs,targets)
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm(net.parameters(), args.max_norm)
        optimizer.step()
        # if args.debug:
        #       print('Batch ID:%d Loss:%f' %(i,loss.data[0]))
        #       continue
        # if step % args.report_every == 0:
        #       cur_loss = eval(net,vocab,val_iter,criterion)
        #       if cur_loss < min_loss:
        #           min_loss = cur_loss
        #           best_path = net.save()

61


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([0.3566, 0.6892, 0.6399, 0.4993, 0.6113, 0.6647, 0.6456, 0.6278, 0.7208,
        0.7855, 0.6755, 0.4561, 0.6992, 0.7255, 0.7829, 0.8333, 0.8704, 0.9035,
        0.7487, 0.8503, 0.6277, 0.8273, 0.7010, 0.5976, 0.7475, 0.8792, 0.6497,
        0.4936, 0.3389, 0.6875, 0.6295, 0.4883, 0.3644, 0.4166, 0.3620, 0.4688,
        0.6242, 0.7073, 0.6563, 0.4761, 0.6289, 0.6496, 0.7326, 0.7948, 0.8087,
        0.8293, 0.6600, 0.7880, 0.6366, 0.8633, 0.8037, 0.6026, 0.6493, 0.8591,
        0.8042, 0.8025, 0.6640, 0.8718, 0.7460, 0.7918, 0.7613, 0.7880, 0.8387,
        0.6427, 0.6069, 0.6630, 0.7292, 0.7727, 0.5579, 0.4574, 0.6639, 0.7164,
        0.9053, 0.8810, 0.7452, 0.8815, 0.7905, 0.7755, 0.8069, 0.8759, 0.7736,
        0.9539, 0.8463, 0.8770, 0.9571, 0.9551, 0.9036, 0.7069, 0.8741, 0.8736,
        0.8422, 0.6299, 0.7847, 0.7314, 0.7561, 0.7640, 0.8261, 0.7142, 0.7635,
        0.7396, 0.7622, 0.7666, 0.9109, 0.9436, 0.7836, 0.6727, 0.6914, 0.6505,
        0.6795, 0.4121, 0.6509, 0.7142, 

RuntimeError: ignored

In [ ]:
 l = [ 4, 25, 24, 36, 31, 16, 26,  1,  8, 20,  6, 15, 15, 12, 13, 13, 12, 31,
        34, 21, 12, 30, 16, 28, 22, 17, 16, 10, 56, 27, 16, 24, 41, 32, 28, 23,
        14,  3, 17, 22, 20, 30, 22, 14,  3, 17, 26, 32, 22, 12,  6,  5, 18, 10,
         3, 23, 30, 11, 20, 32, 13,  2, 25, 31, 24, 21,  3, 25, 10, 20,  9, 41,
        20,  4, 34, 27, 19, 23, 29,  3, 31, 41, 24, 12,  3, 15, 15, 21, 17, 21,
        36, 22, 44, 13,  7, 25,  4, 20, 20, 21, 25,  4,  8, 22,  8,  4,  7,  8,
        11, 14,  9, 14,  7, 33, 20, 12, 15, 50, 27, 11, 22, 26, 30, 24, 29,  3,
        21, 31, 38,  7, 10, 12, 36, 21,  1, 11, 12, 17, 14, 12, 23, 16, 17, 21,
         5,  9,  3, 39, 15,  4, 27, 34, 18, 11, 21, 23, 30, 20, 13, 15, 13, 32,
        40, 17, 18, 12,  4, 12, 27, 32, 17, 33, 12,  2,  6, 18, 19, 14, 24, 33,
        17, 16, 13, 35, 53, 26, 34, 26, 13,  8,  4, 29, 22, 16, 12, 45, 13, 12,
        33,  7, 19, 11, 13, 26,  7,  3, 11, 19, 16, 10, 12, 21, 35, 30, 38, 24,
        27, 15, 19, 39,  5, 12, 35, 20, 11,  8, 10,  3, 24, 23,  6, 21, 20,  7,
        18, 16,  9, 21, 14, 27, 17,  9, 13,  3, 25, 26, 27, 38, 17, 21, 21,  9,
         7, 26, 16,  8, 11, 14,  8,  9,  7, 18,  3, 11, 14, 11,  9,  7,  7, 21,
        12, 10,  2, 18, 12, 25,  5,  8, 22, 16, 17,  8, 10,  6,  7,  5, 18,  5,
         5,  7, 23, 25, 12, 20,  7,  7,  7, 23, 15, 12, 15,  4,  7,  6, 20,  8,
        14, 11, 13, 22,  7,  4,  2, 34, 12, 31,  2, 39, 29,  5, 11,  6,  4, 29,
        28, 23, 16, 29,  8, 17,  9, 24, 41, 20, 36, 34,  8,  8, 19,  1, 17,  3,
        14, 27, 13, 25, 13, 28, 21, 13,  6,  3, 22, 25, 24, 34, 20, 16,  2, 25,
        12, 22, 28, 12, 26,  2, 29, 11, 40, 12, 19,  2, 16, 22, 35, 21, 15,  2,
        21, 25,  9, 17, 14, 17,  7, 41, 28, 38, 14, 11, 22,  4, 25, 27,  9, 25,
        37,  2, 17, 17, 23, 19,  2, 13, 13, 25, 23, 29,  5, 35, 32, 12, 25, 33,
         9, 37, 45, 15, 33, 50, 27, 32, 35, 30, 33, 17, 17, 13, 23,  4, 38, 31,
        29,  5, 44, 28, 30, 47, 46, 17, 45, 23, 29, 21, 17, 25, 51,  7, 12, 34,
        14, 35, 15, 36, 32, 11, 29, 54, 11, 22, 14, 52, 42, 18,  7, 12, 24, 20,
        31, 43, 32,  9, 37, 31, 21, 11, 25, 25, 14, 17, 17, 17, 17, 26, 10,  6,
         9, 14,  8, 26, 10, 23,  3, 24,  1, 30,  7,  3, 29, 30, 18, 12, 17, 24,
         9,  4, 34, 33, 33, 33, 22,  5, 17,  3, 27, 12, 30, 13, 21, 47, 26, 16,
        20, 26,  7, 14, 20, 11, 32, 26, 36, 16, 35, 13, 38, 15, 20, 46, 26, 38,
        29, 13, 33,  8, 11, 17, 33,  9, 21, 22, 28, 16, 28, 32,  3, 46, 33, 15,
        33, 39, 20, 38, 25,  5, 10, 45, 11, 36, 19, 28, 39, 20, 38, 24,  8, 14,
        38, 29, 29, 26,  5, 33, 19, 15, 22, 23, 16, 14, 17, 23, 21, 14, 17, 22,
        13, 12, 38, 45, 23,  4, 31, 18, 17, 16,  9,  6,  4, 19, 41, 27, 28, 45,
        29, 10, 17, 19, 15, 22, 19, 26, 18,  7, 31, 28, 15, 38, 19, 19, 20, 16,
        14, 11, 36, 17, 34, 14, 25, 30, 23, 20, 30, 33, 24, 17, 29, 40, 38, 15,
        21, 18,  8]
len(l)

651

In [ ]:
torch.manual_seed(233)

class SummaRuNNer(nn.Module):
    def __init__(self, config):
        super().__init__()

        # Parameters
        self.vocab_size = config.vocab_size        
        self.embedding_dim = config.embedding_dim
        self.position_size = config.position_size
        self.position_dim = config.position_dim
        self.word_input_size = config.word_input_size
        self.sent_input_size = config.sent_input_size
        self.word_GRU_hidden_units = config.word_GRU_hidden_units
        self.sent_GRU_hidden_units = config.sent_GRU_hidden_units
        
        # Network
        self.word_embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.word_embedding.weight.data.copy_(torch.from_numpy(config.pretrained_embedding))
        self.position_embedding = nn.Embedding(self.position_size, self.position_dim)

        self.word_GRU = nn.GRU(
            input_size = self.word_input_size,
            hidden_size = self.word_GRU_hidden_units,
            batch_first = True,
            bidirectional = True)
        self.sent_GRU = nn.GRU(
            input_size = self.sent_input_size,
            hidden_size = self.sent_GRU_hidden_units,
            batch_first = True,
            bidirectional = True)
        
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
        self.fc1 = nn.Linear(400, 300)
        self.fc2 = nn.Linear(400, 300)
        
        # Parameters of Classification Layer
        self.Wc = Parameter(torch.randn(1, 300))
        self.Ws = Parameter(torch.randn(300, 300))
        self.Wr = Parameter(torch.randn(00, 300))
        self.Wp = Parameter(torch.randn(1, 50))
        self.b = Parameter(torch.randn(1))

    def _avg_pooling(self, x, sequence_length):
        result = []
        for index, data in enumerate(x):
            avg_pooling = torch.mean(data[:sequence_length[index][0], :], dim = 0)
            result.append(avg_pooling)
        return torch.cat(result, dim = 0) 
     
    def forward(self, x):
        sequence_length = torch.sum(torch.sign(x), dim = 1).data
        print(sequence_length)
        sequence_num = sequence_length.size()[0]

        # word level GRU
        word_features = self.word_embedding(x)
        word_outputs, _ = self.word_GRU(word_features)
        # sentence level GRU
        sent_features = self._avg_pooling(word_outputs, sequence_length)
        sent_outputs, _ = self.sent_GRU(sent_features.view(1, -1, self.sent_input_size))
        # document representation
        doc_features = self._avg_pooling(sent_outputs, [[x.size(0)]])
        doc = torch.transpose(self.tanh(self.fc1(doc_features)), 0, 1)
        # classifier layer
        outputs = []
        sent_outputs = sent_outputs.view(-1, 2 * self.sent_GRU_hidden_units)
        
        s = Variable(torch.zeros(100, 1)).cuda()
        
        for position, sent_hidden in enumerate(sent_outputs):
            h = torch.transpose(self.tanh(self.fc2(sent_hidden.view(1, -1))), 0, 1)
            position_index = Variable(torch.LongTensor([[position]])).cuda()
            p = self.position_embedding(position_index).view(-1, 1)
            
            content = torch.mm(self.Wc, h)
            salience = torch.mm(torch.mm(h.view(1, -1), self.Ws), doc)
            novelty = -1 * torch.mm(torch.mm(h.view(1, -1), self.Wr), self.tanh(s))
            position = torch.mm(self.Wp, p)
            bias = self.b
            Prob = self.sigmoid(content + salience + novelty + position + bias)
            s = s + torch.mm(h, Prob)
            outputs.append(Prob)
        
        return torch.cat(outputs, dim = 0)

In [ ]:
weights.shape

(3000002, 300)

In [ ]:
from collections import namedtuple

Config = namedtuple('parameters',
        ['vocab_size', 'embedding_dim', 'position_size','position_dim','word_input_size','sent_input_size',
        'word_GRU_hidden_units','sent_GRU_hidden_units','pretrained_embedding'])

config = Config(
        vocab_size = weights.shape[0],
        embedding_dim = weights.shape[1],
        position_size = 500,
        position_dim = 50,
        word_input_size = 300,
        sent_input_size = 2 * 200,
        word_GRU_hidden_units = 200,
        sent_GRU_hidden_units = 200,
        pretrained_embedding = weights)